In [1]:
from chemtbd.io import Agilent
import numpy as np
import pandas as pd

In [2]:
agi = Agilent.from_root('data')

__matclibarea_one__ matches lib dataframe to fid or tic dataframes

- key: string, typically the file name
- libdf: dataframe, dataframe containing chemical identification information i.e. library/id, rt, etc
- areaddf: dataframe, dataframe containing either tic or fid areas and retetion times (rt)
- areatype: string, `fid` or `tic`

example:
```
key = 'FA13.D'
libdf = agi[key].results.lib
ticdf  = agi[key].results.tic
fiddf = agi[key].results.fid
matchlib2area_one('FA14.D',libdf,ticdf,'tic')
```

__matchlib2area_all__ matches lib dataframe to fid or tic dataframes for all data sets in a folder containing .D files generated from Agilent

example:
```
matched_df = matchlib2area_all(agi,'tic')
```

In [3]:
def matchlib2area_one(key,libdf, areadf,areatype):
    returndf = libdf.copy()
    
    if areatype == 'tic':
        colname = 'tic_area'
    elif areatype == 'fid':
        colname = 'fid_area'
    else:
        print('incorrect area type')

    returndf[colname] = np.nan
    #find shortest
    if len(libdf) <= len(areadf):
        short = libdf
        long = areadf
    else:
        short = areadf
        long = libdf
        
    #loop and match
    match_list = []
    for index,row in short.iterrows():
        sqrError = (short['rt'].ix[index]-long['rt'].ix[index:len(long)])**2    
        area_row = sqrError.idxmin(axis=1)
        match_list.append(areadf['area'].ix[area_row])
    if areatype == 'tic':
        returndf['tic_area'] = match_list
    elif areatype == 'fid':
        returndf['fid_area'] = match_list
    
    return returndf

def matchlib2area_all(data_dic,areatype='tic'):
    libdf = data_dic.results('lib').groupby('key')
    areadf  = data_dic.results(areatype).groupby('key')

    #return_dic = {}
    return_df = pd.DataFrame()
    for name, group in libdf:
        try:
            return_df = return_df.append(matchlib2area_one(name,group,areadf.get_group(name),areatype))
            # return_dic[name] = matchlib2area_one(name,group,areadf.get_group(name),areatype)
        except:
            print(name+' did not match successfully')
    return return_df.reset_index()

matched_df = matchlib2area_all(agi,'tic')
matched_df.head()

FA14_2.D did not match successfully


,index,header=,pk,rt,pct_area,library_id,ref,cas,qual,key,tic_area
0,0,1=,1.0,5.7877,2.0335,Methyl octanoate,17.0,000000-00-0,96.0,FA03.D,1489466.0
1,1,2=,2.0,7.3441,3.4015,Methyl decanoate,1.0,000000-00-0,98.0,FA03.D,2491449.0
2,2,3=,3.0,8.0364,1.7448,Methyl undecanoate,2.0,000000-00-0,98.0,FA03.D,1277982.0
3,3,4=,4.0,8.6715,3.9674,Methyl dodecanoate,3.0,000000-00-0,98.0,FA03.D,2905961.0
4,4,5=,5.0,9.2781,1.9607,Methyl tridecanoate,4.0,000000-00-0,99.0,FA03.D,1436154.0


This creates a df containing the concentrations, I assumed the starting concentration of the stock was 1M, to get from hanna. The general data input represented by __calc_conc_df__ will always be this format; however, we will need to have a different ways for the user to input this information

In [26]:
cal_files = ['FA03.D','FA04.D','FA05.D']
cal_dilutions = {'FA03.D': 0.25,'FA04.D':0.50,'FA05.D':1.0}
cal_conc_df = pd.DataFrame({'library_id':agi['FA03.D'].results['lib']['library_id'],'Conc_0':1})

for key,val in cal_dilutions.items():
    cal_conc_df[key] = cal_conc_df['Conc_0']*val
cal_conc_df.set_index(['library_id'],inplace=True)
cal_conc_df.drop('Conc_0',1,inplace=True)
cal_conc_df.head()

,FA03.D,FA04.D,FA05.D
library_id,,,
Methyl octanoate,0.25,0.5,1.0
Methyl decanoate,0.25,0.5,1.0
Methyl undecanoate,0.25,0.5,1.0
Methyl dodecanoate,0.25,0.5,1.0
Methyl tridecanoate,0.25,0.5,1.0


Calculate a linear fit to calibration data for each species

In [27]:
def calc_calibration(matched_df, cal_conc_df):
    pass

In [28]:
species = 'Methyl octanoate'


Fuctions which still need to be built

In [ ]:
def calc_conc():
    pass

def calc_mass():
    pass

def make_plot_percentarea():
    pass

def make_plot_concarea():
    pass

def make_plot_massarea():
    pass